In [1]:
import tokenize
from io import StringIO

This uses the above libraries to build a Lisp structure based on atoms. It is adapted from [simple iterator parser](http://effbot.org/zone/simple-iterator-parser.htm) (deadlink). The first function is the `atom` function.

In [2]:
def atom( next, token):
    if token[ 1] == '(':
        out = []
        token = next()
        while token[ 1] != ')':
            out.append( atom( next, token))
            token = next()
            if token[ 1] == ' ':
                token = next()
        return out
    elif token[ 1] == '?':
        token = next()
        return "?" + token[ 1]
    else:
        return token[ 1]

The next function is the actual `parse` function:

In [3]:
def parse(exp):
    src = StringIO(exp).readline
    tokens = tokenize.generate_tokens(src)
    return atom(tokens.__next__, tokens.__next__())

From a Python perspective, we want to turn something like "(loves Fred ?x)" to ["loves" "Fred" "?x"] and then work with the second representation as a list of strings. The strings then have the syntactic meaning we gave them previously.

In [4]:
parse("Fred")

'Fred'

In [5]:
parse( "?x")

'?x'

In [6]:
parse( "(loves Fred ?x)")

['loves', 'Fred', '?x']

In [7]:
parse( "(father_of Barney (son_of Barney))")

['father_of', 'Barney', ['son_of', 'Barney']]

In [8]:
parse("((son barney) (son ?x))")

[['son', 'barney'], ['son', '?x']]

## Unifier

Here is imperative pseudocode for the algorithm:

```
def unification( exp1, exp2):
    # base cases
    if exp1 and exp2 are constants or the empty list:
        if exp1 = exp2 then return {}
        else return FAIL
    if exp1 is a variable:
        if exp1 occurs in exp2 then return FAIL
        else return {exp1/exp2}
    if exp2 is a variable:
        if exp2 occurs in exp1 then return FAIL
        else return {exp2/exp1}

    # inductive step
    first1 = first element of exp1
    first2 = first element of exp2
    result1 = unification( first1, first2)
    if result1 = FAIL then return FAIL
    apply result1 to rest of exp1 and exp2
    result2 = unification( rest of exp1, rest of exp2)
    if result2 = FAIL then return FAIL
    return composition of result1 and result2
```

`unification` can return...

1. `None` (if unification completely fails)
2. `{}` (the empty substitution list) or 
3. a substitution list that has variables as keys and substituted values as values, like {"?x": "Fred"}. 

Note that the middle case sometimes confuses people..."Sam" unifying with "Sam" is not a failure so you return {} because there were no variables so there were no substitutions. You do not need to further resolve variables. If a variable resolves to an expression that contains a variable, you don't need to do the substition.

If you think of a typical database table, there is a column, row and value. This Tuple is a *relation* and in some uses of unification, the "thing" in the first spot..."love" above is called the relation. If you have a table of users with user_id, username and the value then the relation is:

`(login ?user_id ?username)`

*most* of the time, the relation name is specified. But it's not impossible for the relation name to be represented by a variable:

`(?relation 12345 "smooth_operator")`

Your code should handle this case (the pseudocode does handle this case so all  you have to do is not futz with it).

Our type system is very simple. We can get by with just a few boolean functions. The first tests to see if an expression is a variable.

In [9]:
def is_variable( exp):
    return isinstance( exp, str) and exp[ 0] == "?"

In [10]:
is_variable( "Fred")

False

In [11]:
is_variable( "?fred")

True

The second tests to see if an expression is a constant:

In [12]:
def is_constant( exp):
    return isinstance( exp, str) and not is_variable( exp)

In [13]:
is_constant( "Fred")

True

In [14]:
is_constant( "?fred")

False

In [15]:
is_constant( ["loves", "Fred", "?wife"])

False

It might also be useful to know that:

<code>
type( "a")
&lt;type 'str'>
type( "a") == str
True
type( "a") == list
False
type( ["a"]) == list
True
</code>

-----

## `occurs`

### Args:
* **var** (*any*): The variable to check for its occurrence within the expression.
* **exp** (*any*): The expression in which to search for the variable. It can be of any type, but typically it is a list or a value.

### Returns
* **result** (*bool*): Returns `True` if `var` occurs within `exp`, or `False` otherwise.


In [28]:
def occurs(var,exp):
    if var == exp:
        return True
    elif isinstance(exp, list):
        for sub_exp in exp:
            if occurs(var,sub_exp):
                return True
        return False
    return False

In [35]:
var1 = '?x'
exp1 = ['?y', '?z', '?x']
expected_result1 = True
assert occurs(var1, exp1) == expected_result1

var2 = 'a'
exp2 = ['b', ['c', 'd'], 'e']
expected_result2 = False
assert occurs(var2, exp2) == expected_result2

var3 = 'var'
exp3 = 'var'
expected_result3 = True
assert occurs(var3, exp3) == expected_result3

## `apply_substitution`

### Args:
* **exp** (*any*): The expression to apply the substitution to. It can be a variable or a list of expressions.
* **substitution** (*dict*): A dictionary representing the substitution, where keys are variables and values are their substitutions.

### Returns
* **result** (*any*): The resulting expression after applying the substitution. If `exp` is a list, the substitution is applied recursively to each element.


In [17]:
def apply_substitution(exp, substitution):
    if is_variable(exp):
        return substitution.get(exp, exp)
    elif isinstance(exp, list):
        result = []
        for sub_exp in exp:
            result.append(apply_substitution(sub_exp, substitution))
        return result
    return exp

In [ ]:
exp1 = 'x'
substitution1 = {'x': 'value'}
expected_result1 = 'value'
assert apply_substitution(exp1, substitution1) == expected_result1

exp2 = 'y'
substitution2 = {'x': 'value'}
expected_result2 = 'y' 
assert apply_substitution(exp2, substitution2) == expected_result2

exp3 = ['x', 'y', 'z']
substitution3 = {'x': 'val1', 'y': 'val2'}
expected_result3 = ['val1', 'val2', 'z'] 
assert apply_substitution(exp3, substitution3) == expected_result3

## `compose`

### Args:
* **substitutions1** (*dict*): The first substitution dictionary where keys are variables and values are their corresponding substitutions.
* **substitutions2** (*dict*): The second substitution dictionary to be applied on top of `substitutions1`.

### Returns
* **composed** (*dict*): A new dictionary representing the composition of `substitutions1` and `substitutions2`, where substitutions from `substitutions2` are applied to the values of `substitutions1`.


In [18]:
def compose(substitutions1:dict, substitutions2:dict) -> dict:
    composed = substitutions1.copy()
    composed.update(substitutions2)
    for var in substitutions1:
        composed[var] = apply_substitution(substitutions1[var], substitutions2)
    return composed

In [44]:
substitutions1 = {'x': 'a'}
substitutions2 = {'y': 'b'}
expected_result1 = {'x': 'a', 'y': 'b'}
assert compose(substitutions1, substitutions2) == expected_result1

substitutions1 = {'x': 'a'}
substitutions2 = {'x': 'b', 'y': 'c'}
expected_result2 = {'x': 'a', 'y': 'c'}
assert (compose(substitutions1, substitutions2))  == expected_result2

substitutions1 = {'x': 'y'}
substitutions2 = {'y': 'z'}
expected_result3 = {'x': 'y', 'y': 'z'}
assert compose(substitutions1, substitutions2) == expected_result3

## `list_check`

### Args:
* **parsed_expression** (*any*): The expression to be checked. It can be any data type, but typically it is either a list or a single value.

### Returns
* **result** (*list*): If `parsed_expression` is already a list, it is returned as is. Otherwise, a new list is created with `parsed_expression` as its only element.


In [19]:
def list_check(parsed_expression):
    if isinstance(parsed_expression, list):
        return parsed_expression
    return [parsed_expression]

In [43]:
parsed_expression1 = ['a', 'b', 'c']
expected_result1 = ['a', 'b', 'c']
assert list_check(parsed_expression1) == expected_result1

parsed_expression2 = 'a'
expected_result2 = ['a']
assert list_check(parsed_expression2) == expected_result2

parsed_expression3 = 42
expected_result3 = [42]
assert list_check(parsed_expression3) == expected_result3

## `unification`

### Args:
* **list_expression1** (*list or any*): The first expression to unify. It can be a list or a single variable.
* **list_expression2** (*list or any*): The second expression to unify. It can be a list or a single variable.

### Returns
* **result** (*dict or None*): Returns a dictionary of substitutions if the unification succeeds. If unification fails, it returns `None`.
    - An empty dictionary `{}` is returned if the two expressions are identical.
    - If one expression is a variable and does not occur in the other, a substitution `{variable: expression}` is returned.
    - If unification is not possible (e.g., occurs check fails, list lengths differ), it returns `None`.


In [20]:
def unification( list_expression1, list_expression2):

    if list_expression1 == list_expression2: 
            return {}
    if is_variable(list_expression1):
        if occurs(list_expression1,list_expression2):
            return None
        return {list_expression1: list_expression2}
    if is_variable(list_expression2):
        if occurs(list_expression2,list_expression1):
            return None
        return {list_expression2: list_expression1}
    if len(list_expression1) == 0 or len(list_expression2) == 0:
        return None
    
    if (isinstance(list_expression1, list) and isinstance(list_expression2,list) and len(list_expression1) == len(list_expression2)):
        first1,rest1 = list_expression1[0], list_expression1[1:]
        first2, rest2 = list_expression2[0], list_expression2[1:]
        result1 = unification(first1, first2)
        if result1 == None:
            return None
        rest1 = apply_substitution(rest1, result1)
        rest2 = apply_substitution(rest2, result1)
        result2 = unification(rest1, rest2)
        if result2 == None: 
            return None 
        return compose(result1,result2)

The `unification` pseudocode only takes lists so we have to make sure that we only pass a list.
However, this has the side effect of making "foo" unify with ["foo"], at the start.
That's ok.

In [21]:
def unify( s_expression1, s_expression2):
    list_expression1 = list_check(parse(s_expression1))
    list_expression2 = list_check(parse(s_expression2))
    return unification( list_expression1, list_expression2)

**Note** If you see the error,

```
tokenize.TokenError: ('EOF in multi-line statement', (2, 0))
```
You most likely have unbalanced parentheses in your s-expression.

## Test Cases

Use the expressions from the Self Check as your test cases...

In [22]:
self_check_test_cases = [
    ['(son Barney Barney)', '(daughter Wilma Pebbles)', None]
]
for case in self_check_test_cases:
    exp1, exp2, expected = case
    actual = unify(exp1, exp2)
    print(f"actual = {actual}")
    print(f"expected = {expected}")
    print("\n")
    assert expected == actual

actual = None
expected = None




In [23]:
new_test_cases = [
    ['(son Barney Barney)', '(daughter Wilma Pebbles)', None, "non-equal constants"], 
    ['(a c b)', '(a c b)', {}, "identical expressions"],
    ['(son ?x)', '(son Augustin)', {'?x': 'Augustin'}, "variable unifies with constant"],
    ['(son ?x brother Barney)', '(son Augustin brother ?y)', {'?x': 'Augustin', '?y':'Barney'}, "variables unifies with constants"],
    ['(son ?x brother ?x)', '(son Augustin brother Barney)', None, "cannot unify to same variable"],
    ['(son ?x brother ?y)', '(son Augustin brother (son Barney))', {'?x':'Augustin', '?y': ['son', 'Barney']}, "variable can unify to compound constant"]
]
for case in new_test_cases:
    exp1, exp2, expected, message = case
    actual = unify(exp1, exp2)
    print(f"Testing {message}...")
    print(f"actual = {actual}")
    print(f"expected = {expected}")
    print("\n")
    assert expected == actual

Testing non-equal constants...
actual = None
expected = None


Testing identical expressions...
actual = {}
expected = {}


Testing variable unifies with constant...
actual = {'?x': 'Augustin'}
expected = {'?x': 'Augustin'}


Testing variables unifies with constants...
actual = {'?x': 'Augustin', '?y': 'Barney'}
expected = {'?x': 'Augustin', '?y': 'Barney'}


Testing cannot unify to same variable...
actual = None
expected = None


Testing variable can unify to compound constant...
actual = {'?x': 'Augustin', '?y': ['son', 'Barney']}
expected = {'?x': 'Augustin', '?y': ['son', 'Barney']}


